In [1]:
import openai
from qdrant_client import QdrantClient

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

/Users/arminaskurmauskas/AI-Engineering_Bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download an example reference data point from LangSmith

In [2]:
client = Client()

In [3]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [4]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('030e9c26-20b4-42c0-843a-d7eccd876f7f'), created_at=datetime.datetime(2026, 1, 18, 9, 58, 42, 45923, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 18, 9, 58, 42, 45923, tzinfo=TzInfo(0)), example_count=43, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-14.6.1-x86_64-i386-64bit', 'sdk_version': '0.6.4', 'runtime_version': '3.12.7', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [23]:
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs
print(reference_output)

{'ground_truth': "From the provided product texts, none explicitly mention wireless charging support for the earbuds' charging cases. They list battery capacities, charging via USB-C or standard charging but do not state wireless charging capability.", 'reference_context_ids': ['B0C6K1GQCF', 'B09WCFC5D9', 'B0CBMPG524', 'B09PRK49JH', 'B0B9FTVL58'], 'reference_descriptions': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the headphones. 

In [24]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs
print(reference_input)

{'question': 'I need to know whether any of your earbuds support wireless charging of the case — can you tell me from the chunks?'}


### RAG Pipeline

In [19]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }


In [ ]:
rag_pipeline("Can you suggest me earbuds for kids?", top_k=5)

{'answer': 'Here are kid-friendly earbuds/headphones from the available products:\n\n- TUNEAKE Kids Headphones (ID: B0C142QS8X) \n  - Over-ear, volume-limited to 94dB, designed for kids, foldable and adjustable for comfort, 3.5mm jack. No microphone, but can use the mic on your connected device. Good for school use and travel.\n\n- QearFun Cat Earbuds for Kids (ID: B0B67ZFRPC)\n  - Wired earbuds with a microphone, kid-sized for school use. Cute cat-ear design and comes with a storage case.\n\n- Open Ear Headphones (ID: B0CBMPG524)\n  - Bluetooth 5.3 open-ear design, safe open-ear fit that doesn’t block the ear canal, with a long battery life and IPX7 waterproof. Suitable for kids who want open-ear comfort and awareness of surroundings.\n\nIf you want specifically wired kid headphones with a mic for online learning, the QearFun Cat Earbuds (B0B67ZFRPC) are a good fit. If you prefer open-ear and wireless, consider the Open Ear Headphones (B0CBMPG524). For a volume-limited option ideal fo

### RAGAS metrics

In [17]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

/var/folders/pn/qzwyly3s1j37xkln4n93x9wh0000gn/T/ipykernel_52246/1973645927.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/var/folders/pn/qzwyly3s1j37xkln4n93x9wh0000gn/T/ipykernel_52246/1973645927.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/var/folders/pn/qzwyly3s1j37xkln4n93x9wh0000gn/T/ipykernel_52246/1973645927.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is depre

In [20]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/var/folders/pn/qzwyly3s1j37xkln4n93x9wh0000gn/T/ipykernel_52246/2270190839.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/pn/qzwyly3s1j37xkln4n93x9wh0000gn/T/ipykernel_52246/2270190839.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [26]:
reference_input

{'question': 'I need to know whether any of your earbuds support wireless charging of the case — can you tell me from the chunks?'}

In [25]:
reference_output

{'ground_truth': "From the provided product texts, none explicitly mention wireless charging support for the earbuds' charging cases. They list battery capacities, charging via USB-C or standard charging but do not state wireless charging capability.",
 'reference_context_ids': ['B0C6K1GQCF',
  'B09WCFC5D9',
  'B0CBMPG524',
  'B09PRK49JH',
  'B0B9FTVL58'],
 'reference_descriptions': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the he

In [27]:
result = rag_pipeline(reference_input["question"])

In [28]:
result

{'answer': 'Based on the available products, there is no explicit mention of wireless charging for the charging case. The descriptions only reference charging case use and battery life, but do not indicate wireless charging support. If you’d like, I can check for more details or suggest models that specifically state wireless charging.',
 'question': 'I need to know whether any of your earbuds support wireless charging of the case — can you tell me from the chunks?',
 'retrieved_context_ids': ['B0B9FTVL58',
  'B0CBMPG524',
  'B0C6K1GQCF',
  'B0B14HTZ59',
  'B0BYYLJRHT'],
 'retrieved_context': ['Wireless Earbuds, Bluetooth 5.3 Headphones with Microphone, 37H Playback LED Power Display, In-Ear Headphones Deep Bass, IPX7 Waterproof, Ultra-Light Earphones with Charging Case, Smart Touch, SportS23-vine earbuds',
  'Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Wa

In [29]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [30]:
await ragas_faithfulness(result, "")

0.6

In [31]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [32]:
await ragas_responce_relevancy(result, "")

np.float64(0.0)

In [33]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [34]:
await ragas_context_precision_id_based(result, reference_output)

0.6

In [35]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [36]:
await ragas_context_recall_id_based(result, reference_output)

0.6